In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sdv
from sdv.evaluation.single_table import evaluate_quality
from sdv.single_table import CTGANSynthesizer
from sdv.single_table import TVAESynthesizer
from sdv.metadata import SingleTableMetadata
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate
from sklearn.base import clone
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
import os

In [4]:
# Global variable
CURR_ACTIVITY = "mmo"
CURR_MUSCLE = "ta_r"
CURR_TYPE = "duration"

In [102]:
# Read the data
all_data = pd.read_excel("Biopak data compiled final 1-70 v2.xlsx")

# Extract just JVA data
n_columns = len(all_data.columns)
step = 14
if CURR_ACTIVITY == "map":
    start = n_columns - step
elif CURR_ACTIVITY == "mle":
    start = n_columns - (step*2)
else:
    start = n_columns - (step*3)
data = all_data.iloc[:,start:start+step].copy()

In [103]:
# Visualize the data
data.head()

,JVA MAP left integral,JVA MAP left Integral <300Hz,JVA MAP left Integral >300Hz,JVA MAP left Integral ratio,JVA MAP left peak amplitude,JVA MAP left peak frequency,JVA MAP left median frequency,JVA MAP right integral,JVA MAP right Integral <300Hz,JVA MAP right Integral >300Hz,JVA MAP right Integral ratio,JVA MAP right peak amplitude,JVA MAP right peak frequency,JVA MAP right median frequency
0,18.2,17.1,1.0,0.06,1.8,24,98,20.6,19.5,1.1,0.06,1.7,20,107
1,21.6,20.4,1.2,0.06,2.0,34,88,36.2,33.5,2.8,0.08,2.5,83,107
2,4.3,3.9,0.4,0.09,0.4,73,83,4.7,4.3,0.4,0.09,0.4,49,88
3,10.1,9.1,1.0,0.11,0.9,78,103,9.7,8.3,1.4,0.17,0.6,78,127
4,8.5,7.4,1.1,0.15,0.6,15,83,76.6,71.0,5.6,0.08,4.4,132,132


In [104]:
def add_quotients(muscle, data):
    intensity_quotients = pd.read_csv(f"quotients/{muscle}/quotient_intensity.csv")
    duration_quotients = pd.read_csv(f"quotients/{muscle}/quotient_duration.csv")
    temp_data_intensity = intensity_quotients.copy()
    temp_data_duration = duration_quotients.copy()
    temp_data_intensity.columns = [f"{muscle}_intensity_" + name
                                for name in temp_data_intensity.columns]
    temp_data_duration.columns = [f"{muscle}_duration_" + name
                                for name in temp_data_duration.columns]
    
    indices_to_insert_nan = [1, 40, 42, 60]

    # Insert NaN values without replacing existing values
    for index in indices_to_insert_nan:
        temp_data_intensity = pd.concat([temp_data_intensity.iloc[:index],
                                        pd.DataFrame([[np.nan] * len(temp_data_intensity.columns)],
                                                    columns=temp_data_intensity.columns),
                                                    temp_data_intensity.iloc[index:]],
                                                    ignore_index=True).copy()
        temp_data_duration = pd.concat([temp_data_duration.iloc[:index],
                                        pd.DataFrame([[np.nan] * len(temp_data_duration.columns)],
                                                    columns=temp_data_duration.columns),
                                                    temp_data_duration.iloc[index:]],
                                                    ignore_index=True).copy()
        
    # Add quotients to JVA data
    new_data = pd.concat([data, temp_data_intensity], axis=1)
    new_data = pd.concat([new_data, temp_data_duration], axis=1)

    return new_data

In [105]:
# Data with quotient
new_data = add_quotients(CURR_ACTIVITY, data)
new_data = new_data.dropna().copy()

In [106]:
new_data.head()

,JVA MAP left integral,JVA MAP left Integral <300Hz,JVA MAP left Integral >300Hz,JVA MAP left Integral ratio,JVA MAP left peak amplitude,JVA MAP left peak frequency,JVA MAP left median frequency,JVA MAP right integral,JVA MAP right Integral <300Hz,JVA MAP right Integral >300Hz,...,map_intensity_mm_r,map_intensity_mm_l,map_intensity_da_r,map_intensity_da_l,map_duration_ta_r,map_duration_ta_l,map_duration_mm_r,map_duration_mm_l,map_duration_da_r,map_duration_da_l
0,18.2,17.1,1.0,0.06,1.8,24,98,20.6,19.5,1.1,...,0.06,0.08,0.09,0.08,70.33,173.33,135.67,96.33,26.67,208.17
2,4.3,3.9,0.4,0.09,0.4,73,83,4.7,4.3,0.4,...,0.06,0.07,0.09,0.08,80.33,183.33,125.67,80.92,16.67,198.17
3,10.1,9.1,1.0,0.11,0.9,78,103,9.7,8.3,1.4,...,0.10,0.11,0.05,0.10,192.00,20.50,43.75,77.67,6.75,189.50
4,8.5,7.4,1.1,0.15,0.6,15,83,76.6,71.0,5.6,...,0.04,0.05,0.07,0.06,141.33,71.17,167.67,122.92,58.67,240.17
5,3.3,2.9,0.4,0.13,0.3,24,73,9.5,8.9,0.6,...,0.07,0.08,0.09,0.08,201.00,201.00,34.75,68.67,1.00,180.50


#### Synthetic data generation

In [107]:
# Function to remove features with 0 variance (constant value features)
def get_const_value_features_to_drop(df):
    return [e for e in df.columns if df[e].nunique() == 1]

def impute_and_remove_zero_var(features_ta_r):
    # Performing imputation to replace any NaN value in the dataset with the median of the feature
    imputer = SimpleImputer(strategy="median")
    features_ta_r_imputed = imputer.fit_transform(features_ta_r)
    features_ta_r_imputed_df = pd.DataFrame(features_ta_r_imputed,
                                            columns=features_ta_r.columns)

    # Remove zero variance features
    columns_to_remove = get_const_value_features_to_drop(features_ta_r_imputed_df)
    features_ta_r_imputed_df.drop(columns=columns_to_remove, inplace=True)

    return features_ta_r_imputed_df

imputed_data = impute_and_remove_zero_var(new_data)

In [108]:
def tune_tvae(features_ta_r_imputed_df, metadata):
    # Tuning a variational autoencoder
    tvae_scores = []
    embedding_dims = [128, 256]
    compress_dims = [128, 256]
    decompress_dims = [128, 256]

    for embedding_dim in embedding_dims:
        for compress_dim in compress_dims:
            for decompress_dim in decompress_dims:
                # Creating a Variational Autoencoder synthesizer
                tvae_synthesizer = TVAESynthesizer(metadata,
                                                embedding_dim=embedding_dim,
                                                compress_dims=(compress_dim,compress_dim),
                                                decompress_dims=(decompress_dim,decompress_dim),
                                                epochs=500)
                
                # Fitting the model
                tvae_synthesizer.fit(features_ta_r_imputed_df)
                
                # Generating synthetic data
                synthetic_data = tvae_synthesizer.sample(num_rows=200)

                # Evaluating synthetic data
                quality_report = evaluate_quality(
                    features_ta_r_imputed_df,
                    synthetic_data,
                    metadata,
                    verbose=False
                )

                tvae_scores.append((quality_report.get_score(), tvae_synthesizer))

    return tvae_scores

def tune_ctgan(features_ta_r_imputed_df, metadata):
    # Tuning a ctgan
    ctgan_scores = []
    embedding_dims = [256, 512]
    generator_dims = [256, 512]
    discriminator_dims = [128, 256]

    for embedding_dim in embedding_dims:
        for generator_dim in generator_dims:
            for discriminator_dim in discriminator_dims:
                # Creating a ctgan synthesizer
                ctgan_synthesizer = CTGANSynthesizer(metadata,
                                                    embedding_dim=embedding_dim,
                                                    generator_dim=(generator_dim,generator_dim),
                                                    discriminator_dim=(discriminator_dim,discriminator_dim),
                                                    epochs=500)
                
                # Fitting the model
                ctgan_synthesizer.fit(features_ta_r_imputed_df)
                
                # Generating synthetic data
                synthetic_data = ctgan_synthesizer.sample(num_rows=200)

                # Evaluating synthetic data
                quality_report = evaluate_quality(
                    features_ta_r_imputed_df,
                    synthetic_data,
                    metadata,
                    verbose=False
                )

                ctgan_scores.append((quality_report.get_score(), ctgan_synthesizer))

    return ctgan_scores

def run_augmentation_pipeline(features_ta_r_imputed_df):

    # Creating metadata object to get metadata about the original dataset of extracted features
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=features_ta_r_imputed_df)

    # Get tvae and ctgan tuning results
    tvae_scores = tune_tvae(features_ta_r_imputed_df, metadata)
    ctgan_scores = tune_ctgan(features_ta_r_imputed_df, metadata)

    # Return scores
    return tvae_scores, ctgan_scores

tvae_scores, ctgan_scores = run_augmentation_pipeline(imputed_data)

In [109]:
# Creating a dataframe of tuning results for all
results = pd.DataFrame({
    "TVAE": sorted([each[0] for each in tvae_scores], reverse=True),
    "CTGAN": sorted([each[0] for each in ctgan_scores], reverse=True),
})
results

,TVAE,CTGAN
0,0.863107,0.802054
1,0.857602,0.770518
2,0.851044,0.763231
3,0.848362,0.760262
4,0.841008,0.748766
5,0.840838,0.723735
6,0.840240,0.702706
7,0.840142,0.701890


In [110]:
# obtaining the best models for each of the six muscles
best_tvae = sorted(tvae_scores, reverse=True)[0]
best_ctgan = sorted(ctgan_scores, reverse=True)[0]

# Save best models
path = "{}_best_models/".format(CURR_ACTIVITY)
if not os.path.exists(path):  
    os.makedirs(path)
best_tvae[1].save(
    filepath = path + "/" + f"tvae.pkl"
)
best_ctgan[1].save(
    filepath = path + "/" + f"ctgan.pkl"
)

In [5]:
# Loading the synthesizer
path = "{}_best_models/".format(CURR_ACTIVITY)
# best_synthesizer = TVAESynthesizer.load(
#     filepath = path + "/" + f"tvae.pkl"
# )
with open(path + "/" + f"tvae.pkl", "rb") as conts_file:
    best_synthesizer = pickle.load(conts_file)

# Generating 2000 synthetic observations using the trained model
synthetic_data = best_synthesizer.sample(num_rows=2000, batch_size=100)

AttributeError: Can't get attribute 'BaseProvider.filter_by_length' on <module 'faker.providers' from '/Users/tashrequehaq/miniconda3/lib/python3.11/site-packages/faker/providers/__init__.py'>

In [14]:
# combine with original data
full_data = pd.concat([synthetic_data, imputed_data], axis=0)

In [15]:
full_data.head()

,JVA MMO left integral,JVA MMO left Integral <300Hz,JVA MMO left Integral >300Hz,JVA MMO left Integral ratio,JVA MMO left peak amplitude,JVA MMO left peak frequency,JVA MMO left median frequency,JVA MMO right integral,JVA MMO right Integral <300Hz,JVA MMO right Integral >300Hz,...,mmo_intensity_mm_r,mmo_intensity_mm_l,mmo_intensity_da_r,mmo_intensity_da_l,mmo_duration_ta_r,mmo_duration_ta_l,mmo_duration_mm_r,mmo_duration_mm_l,mmo_duration_da_r,mmo_duration_da_l
0,3.0,8.2,0.8,0.08,1.7,15.0,79.0,8.0,14.9,0.9,...,0.09,0.09,0.11,0.10,32.25,21.12,29.82,58.28,44.10,62.80
1,20.6,6.7,1.9,0.09,1.3,89.0,101.0,7.7,11.5,1.6,...,0.10,0.09,0.10,0.10,68.27,29.65,82.37,49.73,37.87,61.90
2,16.2,11.7,0.4,0.04,1.1,69.0,76.0,6.7,14.2,2.8,...,0.13,0.19,0.16,0.12,54.60,66.81,46.37,37.73,48.70,25.12
3,13.0,11.4,0.9,0.07,2.1,62.0,86.0,13.9,12.5,0.5,...,0.09,0.10,0.08,0.08,32.53,26.65,132.37,40.51,28.83,38.87
4,4.5,8.9,0.7,0.10,0.3,15.0,112.0,4.3,7.1,0.9,...,0.12,0.14,0.12,0.11,35.08,138.49,17.01,58.46,53.04,62.40


#### Get features and targets

In [16]:
# Extract features (X)
len_columns = len(full_data.columns)
columns = full_data.columns[:len_columns-12]
X = full_data[columns]

In [17]:
X.head()

,JVA MMO left integral,JVA MMO left Integral <300Hz,JVA MMO left Integral >300Hz,JVA MMO left Integral ratio,JVA MMO left peak amplitude,JVA MMO left peak frequency,JVA MMO left median frequency,JVA MMO right integral,JVA MMO right Integral <300Hz,JVA MMO right Integral >300Hz,JVA MMO right Integral ratio,JVA MMO right peak amplitude,JVA MMO right peak frequency,JVA MMO right median frequency
0,3.0,8.2,0.8,0.08,1.7,15.0,79.0,8.0,14.9,0.9,0.06,2.2,26.0,72.0
1,20.6,6.7,1.9,0.09,1.3,89.0,101.0,7.7,11.5,1.6,0.20,1.5,21.0,90.0
2,16.2,11.7,0.4,0.04,1.1,69.0,76.0,6.7,14.2,2.8,0.07,2.1,18.0,96.0
3,13.0,11.4,0.9,0.07,2.1,62.0,86.0,13.9,12.5,0.5,0.09,1.1,19.0,81.0
4,4.5,8.9,0.7,0.10,0.3,15.0,112.0,4.3,7.1,0.9,0.08,0.9,20.0,73.0


In [78]:
# get response variable
y = full_data[f"{CURR_ACTIVITY}_{CURR_TYPE}_{CURR_MUSCLE}"]

#### Splitting the data

In [79]:
# Separating original observations
# original_x = X.iloc[2000:,:]
# original_y = y.iloc[2000:]
# X_temp = X.iloc[:2000,:]
# y_temp = y.iloc[:2000]

X_train = X.iloc[:2000,:].copy()
y_train = y.iloc[:2000].copy()
X_test = X.iloc[2000:,].copy()
y_test = y.iloc[2000:].copy()

# # Creating train test splits
# X_train, X_test, y_train, y_test = train_test_split(
#     X_temp, y_temp, train_size=0.8, random_state=42
# )
# X_test = pd.concat([X_test, original_x]).copy()
# y_test = pd.concat([y_test, original_y]).copy()

In [80]:
# Scaling the data
scaler = StandardScaler()
# scaler = RobustScaler()
X_train_xgb = scaler.fit_transform(X_train).copy()
X_test = scaler.transform(X_test).copy()

#### Modelling

**Linear Regression**

In [81]:
scores_lr = []
# Create a CV instance
cv_lr = KFold(n_splits=5, shuffle=True, random_state=42)
for train, val in cv_lr.split(X_train_xgb, y_train):
    X_train_temp = X_train_xgb[train]
    X_val_temp = X_train_xgb[val]
    y_train_temp = y_train.values[train]
    y_val_temp = y_train.values[val]

    # Define model
    lr_model = LinearRegression()
    lr_model.fit(X_train_temp, y_train_temp)

    # Predictions
    predictions_train = lr_model.predict(X_train_temp)
    predictions_val = lr_model.predict(X_val_temp)

    train_rmse = np.sqrt(mean_squared_error(y_train_temp,
                                            predictions_train))
    val_rmse = np.sqrt(mean_squared_error(y_val_temp,
                                          predictions_val))
    scores_lr.append((val_rmse, train_rmse))

# Compute average scores
avg_val_rmse_lr = np.sum([each[0]
                          for each in scores_lr]) / len(scores_lr)
avg_train_rmse_lr = np.sum([each[1]
                            for each in scores_lr]) / len(scores_lr)

**XGBoost**

In [82]:
# Hyperparameter tuning XGBoost
"""
The following function was taken from
https://xgboost.readthedocs.io/en/stable/python/sklearn_estimator.html
"""
def fit_and_score(estimator, X_train, X_test, y_train, y_test):
    """Fit the estimator on the train set and score it on both sets"""
    estimator.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)], verbose=False)
    predictions_train = estimator.predict(X_train)
    predictions_test = estimator.predict(X_test)
    train_score = np.sqrt(mean_squared_error(y_train,
                                             predictions_train)) #estimator.score(X_train, y_train)
    test_score = np.sqrt(mean_squared_error(y_test,
                                            predictions_test)) #estimator.score(X_test, y_test)
    return estimator, train_score, test_score

# Parameters to tune in the grid
params_to_test = {
    'max_depth': range(3,10,2),
    'min_child_weight': range(1,6,2),
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Performing gridSearch manually
best_xgb_scores = None
best_xgb_model = None
curr_val_score = float('inf')
for depth in params_to_test['max_depth']:
    for weight in params_to_test['min_child_weight']:
        for lr in params_to_test['learning_rate']:
            for estimator_count in params_to_test['n_estimators']:
                # Create a CV instance
                cv = KFold(n_splits=5, shuffle=True, random_state=42)

                # XGBoost model
                xgb_model = XGBRegressor(n_estimators=estimator_count, max_depth=depth,
                                        min_child_weight=weight, early_stopping_rounds=10,
                                        learning_rate=lr)

                # Add cross validation results
                total_train_score = 0
                total_val_score = 0
                for train, val in cv.split(X_train_xgb, y_train):
                    X_train_temp = X_train_xgb[train]
                    X_val_temp = X_train_xgb[val]
                    y_train_temp = y_train.values[train]
                    y_val_temp = y_train.values[val]

                    est, train_score, test_score = fit_and_score(
                        clone(xgb_model), X_train_temp,
                        X_val_temp, y_train_temp,
                        y_val_temp
                    )
                    total_train_score += train_score
                    total_val_score += test_score

                    avg_val_score = total_val_score / 5
                    if avg_val_score < curr_val_score:
                        curr_val_score = avg_val_score
                        best_xgb_scores = (total_train_score / 5, avg_val_score)
                        best_xgb_model = est
                        print(depth, weight, lr, estimator_count)

3 1 0.1 100
3 5 0.1 100


**Random forest**

In [83]:
params_to_test = {
    'max_depth': range(3,10,2),
    'n_estimators': [100, 500, 1000],
    'max_features': [1.0, 'sqrt', 'log2']
}
rf_grid = GridSearchCV(RandomForestRegressor(random_state=42),
                       params_to_test, cv=5, scoring='neg_mean_squared_error',
                       return_train_score=True)

rf_grid.fit(X_train_xgb, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': range(3, 10, 2),
                         'max_features': [1.0, 'sqrt', 'log2'],
                         'n_estimators': [100, 500, 1000]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [84]:
rf_train_score = np.sqrt(
    -rf_grid.cv_results_['mean_train_score'][rf_grid.best_index_])
rf_val_score = np.sqrt(-rf_grid.best_score_)

#### Save validation results

In [85]:
val_results = {f"best_train_rmse_xgboost":[best_xgb_scores[0]],
               f"best_val_rmse_xgboost":[best_xgb_scores[1]],
               f"best_train_rmse_random_forest":[rf_train_score],
               f"best_val_rmse_random_forest":[rf_val_score],
               f"avg_train_rmse_linear_regression":[avg_train_rmse_lr],
               f"avg_val_rmse_linear_regression":[avg_val_rmse_lr]}
val_df = pd.DataFrame(val_results)
val_df.index = [CURR_MUSCLE]
val_df.to_csv(f"obj1_results/{CURR_ACTIVITY}/val_results_{CURR_TYPE}.csv",
              mode="a")

#### Test results

In [86]:
# Best model was XGBoost
test_predictions = best_xgb_model.predict(X_test)

# Performance metrics
test_mae = mean_absolute_error(y_test, test_predictions)
test_rmse = np.sqrt(mean_squared_error(y_test,
                                       test_predictions))
test_mae, test_rmse

(0.03075124128749877, 0.03959518599850674)

In [87]:
# Save results
test_results = {f"test_rmse_xgboost":[test_rmse],
                f"test_mae_xgboost":[test_mae]}
test_df = pd.DataFrame(test_results)
test_df.index = [CURR_MUSCLE]
test_df.to_csv(f"obj1_results/{CURR_ACTIVITY}/test_results_{CURR_TYPE}.csv",
               mode="a")